In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import itertools
import sys

def is_independent_set(G, nodes):
    """
    Checks if the given set of nodes forms an Independent Set
    (no two nodes in the set are connected).
    """
    return G.subgraph(nodes).number_of_edges() == 0

def check_phi_n(G, n):
    """
    Verifies if Graph G satisfies property Phi_n.
    
    Phi_n: For any independent set V = {v1...vn} and any set W = {w1...wn}
    (disjoint from V), there exists a vertex z (not in V u W) such that:
    z connects to ALL v in V AND z connects to NO w in W.
    """
    nodes = set(G.nodes())
    
    # 1. Iterate over all possible sets V of size n
    for v_tuple in itertools.combinations(nodes, n):
        v_set = set(v_tuple)
        
        # Optimization: The definition requires V to be an Independent Set.
        # If V has edges, we skip this tuple (condition vacuously true or N/A).
        if not is_independent_set(G, v_set):
            continue
            
        # 2. Iterate over all possible sets W of size n disjoint from V
        remaining_nodes = nodes - v_set
        for w_tuple in itertools.combinations(remaining_nodes, n):
            w_set = set(w_tuple)
            
            # 3. Search for a witness z
            # Potential z must be outside V and W
            candidates = remaining_nodes - w_set
            
            witness_found = False
            for z in candidates:
                neighbors_of_z = set(G.neighbors(z))
                
                # Check connection to ALL v
                connected_to_all_v = v_set.issubset(neighbors_of_z)
                
                # Check connection to NO w
                connected_to_no_w = w_set.isdisjoint(neighbors_of_z)
                
                if connected_to_all_v and connected_to_no_w:
                    witness_found = True
                    break
            
            # If for this specific pair (V, W), no z exists, the graph fails.
            if not witness_found:
                return False

    # If we survive all combinations, the graph has the property.
    return True

def find_graph_with_property(n, start_size=10, max_size=30, p=0.38, max_attempts=1000000):
    """
    Monte Carlo simulation to find a graph satisfying Phi_n.
    """
    print(f"--- Searching for graph satisfying Phi_{n} ---")
    
    # Heuristic: Start searching at a size where existence is plausible.
    # For n=1, size 4-5 is fine. For n=2, we usually need N >= 10.
    # For n=3, N often needs to be > 20, which gets very slow.
    
    for N in range(start_size, max_size + 1):
        print(f"Checking graphs of size N={N}...")
        
        for i in range(max_attempts):
            # Generate Random Graph G(N, p) (Erdos-Renyi)
            G = nx.erdos_renyi_graph(N, p)
            
            if check_phi_n(G, n):
                print(f"Success! Found a graph at N={N}, attempt {i+1}")
                return G
                
        print(f"No graph found for N={N} after {max_attempts} attempts.")
    
    print("Could not find a graph within constraints.")
    return None

# --- Main Execution ---
if __name__ == "__main__":
    # WARNING: Complexity scales exponentially with n. 
    # n=1 is trivial. n=2 is fast. n=3 is very slow.
    target_n = 2
    
    # We start N slightly higher to increase probability of success
    # Heuristic lower bound usually helps avoid wasting time on tiny graphs
    min_vertices = target_n * 3 
    
    result_graph = find_graph_with_property(target_n, start_size=min_vertices)
    
    if result_graph:
        print("\nAdjacency List of the found graph:")
        for line in nx.generate_adjlist(result_graph):
            print(line)
            
        print(f"\nGraph Order: {result_graph.order()}")
        print(f"Graph Size (Edges): {result_graph.size()}")
        
        nx.draw(result_graph, with_labels=True, node_color='skyblue', edge_color='gray', node_size=1000, font_size=15)
        plt.show()


        # Optional: verify specifically for one case to show it works
        # This is just a sanity check print
        print("\nVerification check complete. This graph satisfies the property.")

--- Searching for graph satisfying Phi_2 ---
Checking graphs of size N=6...
No graph found for N=6 after 1000000 attempts.
Checking graphs of size N=7...
No graph found for N=7 after 1000000 attempts.
Checking graphs of size N=8...
No graph found for N=8 after 1000000 attempts.
Checking graphs of size N=9...
No graph found for N=9 after 1000000 attempts.
Checking graphs of size N=10...
No graph found for N=10 after 1000000 attempts.
Checking graphs of size N=11...
No graph found for N=11 after 1000000 attempts.
Checking graphs of size N=12...
No graph found for N=12 after 1000000 attempts.
Checking graphs of size N=13...
No graph found for N=13 after 1000000 attempts.
Checking graphs of size N=14...
No graph found for N=14 after 1000000 attempts.
Checking graphs of size N=15...
No graph found for N=15 after 1000000 attempts.
Checking graphs of size N=16...
No graph found for N=16 after 1000000 attempts.
Checking graphs of size N=17...
No graph found for N=17 after 1000000 attempts.
Che